  <img width="800px" src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/main/images/fsi/smart-claims/rule_engine.png" />

In [0]:
%sql
CREATE OR REPLACE TABLE smart_claims_dev.03_gold.customer_claim_policy_telematics_predicted AS 
  SELECT
    t.*,
    a.* EXCEPT (chassis_no, claim_no)
  FROM
    smart_claims_dev.03_gold.customer_claim_policy_telematics t
    JOIN smart_claims_dev.03_gold.claim_images_predicted a USING(claim_no)

In [0]:
%sql
CREATE OR REPLACE TABLE smart_claims_dev.02_silver.claims_rules (
  rule_id BIGINT GENERATED ALWAYS AS IDENTITY,
  rule STRING, 
  check_name STRING,
  check_code STRING,
  check_severity STRING,
  is_active Boolean
);

In [0]:
def insert_rule(rule, name, code, severity, is_active):
  spark.sql(f"INSERT INTO smart_claims_dev.02_silver.claims_rules(rule,check_name, check_code, check_severity,is_active) values('{rule}', '{name}', '{code}', '{severity}', {is_active})")

In [0]:
invalid_policy_date = '''
  CASE WHEN to_date(pol_eff_date, "yyyy-MM-dd") < to_date(claim_date) and to_date(pol_expiry_date, "yyyy-MM-dd") < to_date(claim_date) THEN "VALID" 
  ELSE "NOT VALID"  
  END
'''

insert_rule('invalid policy date', 'valid_date', invalid_policy_date, 'HIGH', True)

In [0]:
exceeds_policy_amount = '''
CASE WHEN  sum_insured >= total 
    THEN "claim value in the range of premium"
    ELSE "claim value more than premium"
END 
'''
insert_rule('exceeds policy amount', 'valid_amount', exceeds_policy_amount, 'HIGH', True)

In [0]:
severity_mismatch = '''
CASE WHEN    incident_severity="Total Loss" AND damage_prediction.label = "major" THEN  "Severity matches the report"
       WHEN  incident_severity="Major Damage" AND damage_prediction.label = "minor"  THEN  "Severity matches the report"
       WHEN  (incident_severity="Minor Damage" OR incident_severity="Trivial Damage") AND damage_prediction.label = "ok" THEN  "Severity matches the report"
       ELSE "Severity does not match"
END 
'''

insert_rule('severity mismatch', 'reported_severity_check', severity_mismatch, 'HIGH', True)

In [0]:
exceeds_speed = '''
CASE WHEN  telematics_speed <= 45 and telematics_speed > 0 THEN  "Normal Speed"
       WHEN telematics_speed > 45 THEN  "High Speed"
       ELSE "Invalid speed"
END
'''
insert_rule('exceeds speed', 'speed_check', exceeds_speed, 'HIGH', True)

In [0]:
release_funds = '''
CASE WHEN  reported_severity_check="Severity matches the report" and valid_amount="claim value in the range of premium" and valid_date="VALID" then "release funds"
       ELSE "claim needs more investigation" 
END
'''
insert_rule('release funds', 'fund_release', release_funds, 'HIGH', True)

In [0]:
from pyspark.sql.functions import expr

df = spark.sql("SELECT * FROM smart_claims_dev.03_gold.customer_claim_policy_telematics_predicted")
rules = spark.sql('SELECT * FROM smart_claims_dev.02_silver.claims_rules where is_active=true order by rule_id').collect()
for rule in rules:
  print(rule.rule, rule.check_code)
  df=df.withColumn(rule.check_name, expr(rule.check_code))

#overwrite table with new insights
df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("smart_claims_dev.03_gold.claim_insights")

In [0]:
display(rules)

In [0]:
%sql
SELECT *  FROM smart_claims_dev.03_gold.claim_insights
LIMIT 1